In [1]:
import numpy as np
import time
x_train=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/dmnist10kfromsenet.npy')
x_train=x_train.reshape(10000,2000)
y=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/lmnist10kfromsenet.npy')
y=y.reshape(10000,)
alpha_trans1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/cknnmnist10kfromsenet.npy',allow_pickle=True)
alpha_trans1=np.asarray(alpha_trans1).astype('float32') 
print(x_train.shape,y.shape,alpha_trans1.shape)

(10000, 2000) (10000,) (20, 500, 500)


In [2]:
!pip install memory_profiler
%load_ext memory_profiler

Defaulting to user installation because normal site-packages is not writeable


In [3]:
%memit

peak memory: 160.56 MiB, increment: 0.04 MiB


In [3]:
import glob
import numpy as np
import random
import tensorflow.compat.v1 as tf
import os
from sklearn.utils import shuffle
batchSize=500
from sklearn.metrics.pairwise import euclidean_distances
from scipy.linalg import fractional_matrix_power
from scipy import linalg
import sklearn
from sklearn.cluster import KMeans
from tensorflow.keras import backend as B
k_cluster = 10
from tensorflow.keras.layers import Input,Dense,Lambda,Layer
from tensorflow.keras.models import Model
import sys
import numpy as np
from scipy.io import loadmat
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from munkres import Munkres

from numpy import save


def eig_vec(x_train):
    nev_trans=[]
    SortedEigVec=np.zeros((batchSize,batchSize))
    for l in range (0,len(x_train),batchSize):
        xtr=x_train[l:l+batchSize]
        d=(euclidean_distances(xtr,xtr))
        W=np.zeros((len(xtr),len(xtr)))
        for i in range(len(xtr)):
            for j in range (len(xtr)):
                if i==j:
                    W[i,j]=0
                else:
                    W[i,j]=np.exp(-(d[i,j]**2)/(2*0.09))
        degree=np.diag(sum(W))
        L = degree-W
        eig_val,eig_vec=np.linalg.eig(L)
        dec_eigVal=np.flip(np.sort(eig_val))
        indexArr=(np.flip(np.argsort(eig_val)))
        for i in range (len(indexArr)):
            SortedEigVec[:,i]=eig_vec[:,indexArr[i]]
        SortedEigVec=np.array(SortedEigVec)
        nev_trans.append(SortedEigVec)
    return nev_trans,L,eig_val,eig_vec,dec_eigVal,W

def make_cost_matrix(c1, c2):
    uc1 = np.unique(c1)
    uc2 = np.unique(c2)
    l1 = uc1.size
    l2 = uc2.size
    #print('uc1 shape is',l1)
    #print('uc2 shape is',l2)
    
    ## if 10==10 return true else return false
    assert(l1 == l2 and np.all(uc1 == uc2))
    m = np.ones([l1, l2])
    for i in range(l1):
        it_i = np.nonzero(c1 == uc1[i])[0]
        for j in range(l2):
            it_j = np.nonzero(c2 == uc2[j])[0]
            m_ij = np.intersect1d(it_j, it_i)
            m[i,j] =  -m_ij.size
    return m

def translate_clustering(clt, mapper):
    return np.array([ mapper[i] for i in clt ])

def accuracy(cm):
    return np.trace(cm, dtype=float) / np.sum(cm)

def rectify_label(labels, classes):
    num_labels = len(np.unique(classes))
    cm = confusion_matrix(classes, labels, labels=range(num_labels)) # gets the confusion matrix
    cost_matrix = make_cost_matrix(labels, classes) 
    m = Munkres()
    indexes = m.compute(cost_matrix)
    mapper = { old: new for (old, new) in indexes }
    new_labels = translate_clustering(labels, mapper)
    new_cm = confusion_matrix(classes, new_labels, labels=range(num_labels))
    return new_labels,accuracy(new_cm)


def arch():
    encoding_dim = 490 
    input_img = Input(shape=(2000,))
    encoded_layer_2 = Dense(1500, activation='tanh')(input_img)
    encoded_layer_3 = Dense(1000, activation='tanh')(encoded_layer_2)
    encoded_layer_4 = Dense(700, activation='tanh')(encoded_layer_3)
    encoded = Dense(encoding_dim, activation='tanh')(encoded_layer_4)
    decoded_layer_2 = Dense(700, activation='tanh')(encoded)
    decoded_layer_3 = Dense(1000, activation='tanh')(decoded_layer_2)
    decoded_layer_4 = Dense(1500, activation='tanh')(decoded_layer_3)
    decoded = Dense(2000, activation='tanh')(decoded_layer_4)
    encoder=Model(input_img,encoded)
    autoencoder=Model(input_img,decoded)
    return encoder,encoding_dim,autoencoder

def assign_func(z_test,z_ic):
    dst = np.array(np.sum(((z_test-z_ic[0]) **2),axis=1))
    for index in range(1, z_ic.shape[0]):
        col = np.sum(((z_test- z_ic[index])**2), axis=1)
        dst = np.vstack((dst, col))
    dst = dst.T      
    
    mean_dst = np.mean(dst, axis=1)
    q = np.maximum(0.0, np.tile(mean_dst, (dst.shape[1], 1)).T - dst)
    #print(q.shape)
    num_centers = q.shape[1]
    weight = 1.0 / (q.sum(axis=0) + 1e-9)
    weight *= num_centers / weight.sum()
    q = (q ** 2.0) * weight
    q = (q.T / (q.sum(axis=1) + B.epsilon())).T
    return q

def asl():
    def myloss(y_true, y_pred):
        l1norm= l1norm_func(y_pred,y_true)
        l2norm= l2norm_func(y_pred,y_true)
        orthonormal= orthonormality_func(y_pred)
        structpreserve=structurePreserve_func(y_pred,alpha_trans1[g][:,:])
        return structpreserve+l1norm+l2norm+orthonormal
    return myloss

def l1norm_func(z,nt):
    q1=0.002*(tf.reduce_sum(tf.abs(tf.subtract(z,nt)),axis=1))
    return q1

def l2norm_func(z,nt):
    q2=0.02*(tf.sqrt(tf.reduce_sum(tf.square(z-nt))))
    return q2

def structurePreserve_func(z,ta):
    tz=tf.transpose(z)
    tz=tf.reshape(tz,(encoding_dim,batchSize))
    taa=tf.transpose(ta)
    taz=tf.matmul(tz,taa)
    taz=tf.reshape(taz,(encoding_dim,batchSize))
    q4=0.02*(tf.sqrt(tf.reduce_sum(tf.square(tz-taz))))
    q4=tf.transpose(q4)
    return q4

def orthonormality_func(z):
    z=[(z[i,:]/(tf.norm(z[i,:],2)))for i in range (batchSize)]
    transz=tf.transpose(z)
    q3=tf.tensordot(transz,z,axes=1)
    I=tf.eye(encoding_dim)
    q5=0.002*(tf.sqrt(tf.reduce_sum(tf.square(q3-I))))
    return q5



In [5]:
nev_trans,L,eig_val,eig_vec,dec_eigVal,W=eig_vec(x_train)

In [6]:
%memit

peak memory: 485.68 MiB, increment: 0.34 MiB


In [7]:
startae = time.time()
encoder,encoding_dim,autoencoder=arch()
autoencoder.compile(tf.keras.optimizers.Adam(lr=1e-3), loss=tf.keras.losses.MeanSquaredError())
autoencoder.fit(x_train,x_train,epochs=100,shuffle=True,batch_size=batchSize)
encoder.save_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/AE_mnist10k.ckpt')
endae=time.time()
print('AE training time is : ',endae - startae)

print('training from scratch')
encoder.load_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/AE_mnist10k.ckpt')
g=0
encoder.compile(tf.keras.optimizers.Adadelta(lr=1e-3, rho=0.95, decay=1e-4), loss=asl())
%memit


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 10000 samples
Epoch 1/100
10000/10000 [==============================] - 6s 597us/sample - loss: 5.1370e-04
Epoch 2/100
10000/10000 [==============================] - 5s 505us/sample - loss: 2.4384e-04
Epoch 3/100
10000/10000 [==============================] - 5s 495us/sample - loss: 1.6834e-04
Epoch 4/100
10000/10000 [==============================] - 5s 510us/sample - loss: 1.3576e-04
Epoch 5/100
10000/10000 [==============================] - 5s 507us/sample - loss: 1.1718e-04
Epoch 6/100
10000/10000 [==============================] - 5s 501us/sample - loss: 1.0413e-04
Epoch 7/100
10000/10000 [==============================] - 5s 465us/sample - loss: 9.4490e-05
Epoch 8/100
10000/10000 [==============================] - 5s 509us/sample - loss: 8.7009e-05
Epoch 9/100
10000/10000 [==============================] - 5s 520us/sample - loss: 8.0999e-05
Epoch 10/100
10000/10000 [========================

10000/10000 [==============================] - 5s 532us/sample - loss: 2.3633e-05
Epoch 84/100
10000/10000 [==============================] - 5s 549us/sample - loss: 2.3491e-05
Epoch 85/100
10000/10000 [==============================] - 6s 574us/sample - loss: 2.3331e-05
Epoch 86/100
10000/10000 [==============================] - 6s 584us/sample - loss: 2.3182e-05
Epoch 87/100
10000/10000 [==============================] - 6s 567us/sample - loss: 2.3064e-05
Epoch 88/100
10000/10000 [==============================] - 6s 552us/sample - loss: 2.2951e-05
Epoch 89/100
10000/10000 [==============================] - 5s 544us/sample - loss: 2.2879e-05
Epoch 90/100
10000/10000 [==============================] - 5s 550us/sample - loss: 2.2731e-05
Epoch 91/100
10000/10000 [==============================] - 5s 536us/sample - loss: 2.2592e-05
Epoch 92/100
10000/10000 [==============================] - 6s 558us/sample - loss: 2.2500e-05
Epoch 93/100
10000/10000 [==============================] - 5s 

In [8]:
start = time.time()
maxnum=0
for t in range (1,300,1):
    g=0
    for l in range (0,len(x_train),batchSize):
        encoder.fit(x_train[l:l+batchSize],nev_trans[g][:,batchSize-encoding_dim-1:batchSize-1],epochs=1,shuffle=False,batch_size=batchSize)
        g=g+1
    z_test=encoder.predict(x_train)
    k_means = KMeans(n_clusters=k_cluster, init='k-means++')
    ic= k_means.fit(z_test).cluster_centers_ 
    assignment = assign_func(z_test, ic)
    predictedlabels = np.argmax(assignment, axis=1)
    rectified_label, new_acc=rectify_label(predictedlabels, y)
    print('acc at',t, 'epoch is:',maxnum)
    if maxnum<new_acc:
        maxnum=new_acc
        encoder.save_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/sdsc_mnist10k.ckpt')
        np.save('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/mnistpca2000/sdsc_mnist10k.npy',ic)
        maxnmi=sklearn.metrics.normalized_mutual_info_score(y, predictedlabels)
        maxf1=sklearn.metrics.f1_score(y, rectified_label,average='micro')
        maxprec=sklearn.metrics.precision_score(y, rectified_label,average='weighted')
    else:
        continue
end=time.time()
print('E training time is : ',end - start)

Train on 500 samples
500/500 [==============================] - 17s 34ms/sample - loss: 1.4042
Train on 500 samples
500/500 [==============================] - 0s 932us/sample - loss: 1.4284
Train on 500 samples
500/500 [==============================] - 0s 751us/sample - loss: 1.4228
Train on 500 samples
500/500 [==============================] - 0s 785us/sample - loss: 1.4093
Train on 500 samples
500/500 [==============================] - 0s 734us/sample - loss: 1.4213
Train on 500 samples
500/500 [==============================] - 0s 836us/sample - loss: 1.4181
Train on 500 samples
500/500 [==============================] - 0s 746us/sample - loss: 1.4143
Train on 500 samples
500/500 [==============================] - 0s 747us/sample - loss: 1.4172
Train on 500 samples
500/500 [==============================] - 0s 791us/sample - loss: 1.4105
Train on 500 samples
500/500 [==============================] - 0s 824us/sample - loss: 1.4240
Train on 500 samples
500/500 [====================

Train on 500 samples
500/500 [==============================] - 0s 644us/sample - loss: 1.3941
Train on 500 samples
500/500 [==============================] - 0s 688us/sample - loss: 1.3967
Train on 500 samples
500/500 [==============================] - 0s 674us/sample - loss: 1.3902
Train on 500 samples
500/500 [==============================] - 0s 761us/sample - loss: 1.4038
Train on 500 samples
500/500 [==============================] - 0s 740us/sample - loss: 1.3876
Train on 500 samples
500/500 [==============================] - 0s 665us/sample - loss: 1.3935
Train on 500 samples
500/500 [==============================] - 0s 699us/sample - loss: 1.4123
Train on 500 samples
500/500 [==============================] - 0s 676us/sample - loss: 1.4051
Train on 500 samples
500/500 [==============================] - 0s 785us/sample - loss: 1.4011
Train on 500 samples
500/500 [==============================] - 0s 745us/sample - loss: 1.3982
Train on 500 samples
500/500 [====================

500/500 [==============================] - 0s 553us/sample - loss: 1.3751
Train on 500 samples
500/500 [==============================] - 0s 590us/sample - loss: 1.3940
Train on 500 samples
500/500 [==============================] - 0s 636us/sample - loss: 1.3873
Train on 500 samples
500/500 [==============================] - 0s 596us/sample - loss: 1.3833
Train on 500 samples
500/500 [==============================] - 0s 629us/sample - loss: 1.3800
Train on 500 samples
500/500 [==============================] - 0s 615us/sample - loss: 1.3703
Train on 500 samples
500/500 [==============================] - 0s 643us/sample - loss: 1.3852
Train on 500 samples
500/500 [==============================] - 0s 653us/sample - loss: 1.3832
Train on 500 samples
500/500 [==============================] - 0s 559us/sample - loss: 1.3847
acc at 9 epoch is: 0.8128
Train on 500 samples
500/500 [==============================] - 0s 621us/sample - loss: 1.3610
Train on 500 samples
500/500 [===============

Train on 500 samples
500/500 [==============================] - 0s 594us/sample - loss: 1.3695
Train on 500 samples
500/500 [==============================] - 0s 581us/sample - loss: 1.3668
Train on 500 samples
500/500 [==============================] - 0s 692us/sample - loss: 1.3685
acc at 13 epoch is: 0.8128
Train on 500 samples
500/500 [==============================] - 0s 611us/sample - loss: 1.3447
Train on 500 samples
500/500 [==============================] - 0s 621us/sample - loss: 1.3670
Train on 500 samples
500/500 [==============================] - 0s 651us/sample - loss: 1.3655
Train on 500 samples
500/500 [==============================] - 0s 671us/sample - loss: 1.3497
Train on 500 samples
500/500 [==============================] - 0s 721us/sample - loss: 1.3631
Train on 500 samples
500/500 [==============================] - 0s 698us/sample - loss: 1.3587
Train on 500 samples
500/500 [==============================] - 0s 852us/sample - loss: 1.3553
Train on 500 samples
50

Train on 500 samples
500/500 [==============================] - 0s 622us/sample - loss: 1.3513
Train on 500 samples
500/500 [==============================] - 0s 604us/sample - loss: 1.3348
Train on 500 samples
500/500 [==============================] - 0s 739us/sample - loss: 1.3486
Train on 500 samples
500/500 [==============================] - 0s 641us/sample - loss: 1.3440
Train on 500 samples
500/500 [==============================] - 0s 666us/sample - loss: 1.3406
Train on 500 samples
500/500 [==============================] - 0s 561us/sample - loss: 1.3429
Train on 500 samples
500/500 [==============================] - 0s 678us/sample - loss: 1.3367
Train on 500 samples
500/500 [==============================] - 0s 632us/sample - loss: 1.3505
Train on 500 samples
500/500 [==============================] - 0s 604us/sample - loss: 1.3359
Train on 500 samples
500/500 [==============================] - 0s 689us/sample - loss: 1.3399
Train on 500 samples
500/500 [====================

...

Train on 500 samples
500/500 [==============================] - 0s 840us/sample - loss: 1.0815
Train on 500 samples
500/500 [==============================] - 0s 836us/sample - loss: 1.0765
Train on 500 samples
500/500 [==============================] - 0s 915us/sample - loss: 1.0694
Train on 500 samples
500/500 [==============================] - 0s 830us/sample - loss: 1.0760
Train on 500 samples
500/500 [==============================] - 0s 886us/sample - loss: 1.0702
Train on 500 samples
500/500 [==============================] - 0s 731us/sample - loss: 1.0826
Train on 500 samples
500/500 [==============================] - 0s 748us/sample - loss: 1.0732
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0730
Train on 500 samples
500/500 [==============================] - 0s 878us/sample - loss: 1.0906
Train on 500 samples
500/500 [==============================] - 0s 776us/sample - loss: 1.0873
Train on 500 samples
500/500 [======================

500/500 [==============================] - 0s 896us/sample - loss: 1.0717
Train on 500 samples
500/500 [==============================] - 0s 760us/sample - loss: 1.0714
Train on 500 samples
500/500 [==============================] - 0s 795us/sample - loss: 1.0890
Train on 500 samples
500/500 [==============================] - 0s 818us/sample - loss: 1.0858
Train on 500 samples
500/500 [==============================] - 0s 918us/sample - loss: 1.0809
Train on 500 samples
500/500 [==============================] - 0s 706us/sample - loss: 1.0775
Train on 500 samples
500/500 [==============================] - 0s 728us/sample - loss: 1.0694
Train on 500 samples
500/500 [==============================] - 0s 705us/sample - loss: 1.0899
Train on 500 samples
500/500 [==============================] - 0s 673us/sample - loss: 1.0800
Train on 500 samples
500/500 [==============================] - 0s 670us/sample - loss: 1.0808
acc at 284 epoch is: 0.9599
Train on 500 samples
500/500 [=============

Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0883
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0785
Train on 500 samples
500/500 [==============================] - 0s 985us/sample - loss: 1.0793
acc at 288 epoch is: 0.9599
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0589
Train on 500 samples
500/500 [==============================] - 0s 992us/sample - loss: 1.0783
Train on 500 samples
500/500 [==============================] - 0s 941us/sample - loss: 1.0888
Train on 500 samples
500/500 [==============================] - 0s 972us/sample - loss: 1.0587
Train on 500 samples
500/500 [==============================] - 0s 781us/sample - loss: 1.0780
Train on 500 samples
500/500 [==============================] - 0s 820us/sample - loss: 1.0730
Train on 500 samples
500/500 [==============================] - 0s 811us/sample - loss: 1.0658
Train on 500 samples
500/500

Train on 500 samples
500/500 [==============================] - 0s 892us/sample - loss: 1.0572
Train on 500 samples
500/500 [==============================] - 0s 877us/sample - loss: 1.0765
Train on 500 samples
500/500 [==============================] - 0s 922us/sample - loss: 1.0715
Train on 500 samples
500/500 [==============================] - 0s 943us/sample - loss: 1.0643
Train on 500 samples
500/500 [==============================] - 0s 919us/sample - loss: 1.0711
Train on 500 samples
500/500 [==============================] - 0s 935us/sample - loss: 1.0653
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0776
Train on 500 samples
500/500 [==============================] - 0s 945us/sample - loss: 1.0683
Train on 500 samples
500/500 [==============================] - 0s 963us/sample - loss: 1.0680
Train on 500 samples
500/500 [==============================] - 0s 956us/sample - loss: 1.0856
Train on 500 samples
500/500 [======================

Train on 500 samples
500/500 [==============================] - 0s 866us/sample - loss: 1.0761
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0668
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0665
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0841
Train on 500 samples
500/500 [==============================] - 1s 1ms/sample - loss: 1.0808
Train on 500 samples
500/500 [==============================] - 0s 924us/sample - loss: 1.0760
Train on 500 samples
500/500 [==============================] - 0s 905us/sample - loss: 1.0725
Train on 500 samples
500/500 [==============================] - 0s 832us/sample - loss: 1.0645
Train on 500 samples
500/500 [==============================] - 0s 827us/sample - loss: 1.0850
Train on 500 samples
500/500 [==============================] - 0s 885us/sample - loss: 1.0752
Train on 500 samples
500/500 [============================

In [9]:
%memit

peak memory: 2408.45 MiB, increment: 0.39 MiB


In [10]:
print(maxnum,maxnmi,maxf1,maxprec)

0.9599 0.9002207696393617 0.9599 0.9602290292120677


In [4]:
import numpy as np
import time
x1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/dmnist70kfromsenet.npy')
x1=x1.reshape(70000,2000)
x_test=x1[10000:,:]
y1=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/lmnist70kfromsenet.npy')
y1=y1.reshape(70000,)
y_test=y1[10000:,]

start=time.time()
encoder,encoding_dim,autoencoder=arch()
encoder.load_weights('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/MNIST/sdsc_mnist10k.ckpt')
ic=np.load('/media/itu-cvl/New Volume/DSE paper/literature/SeNet/datasets/mnistpca2000/sdsc_mnist10k.npy')
z_test=encoder.predict(x_test)
assignment = assign_func(z_test, ic)
predictedlabels = np.argmax(assignment, axis=1)
rectified_label, acc=rectify_label(predictedlabels, y_test)
nmi=sklearn.metrics.normalized_mutual_info_score(y_test, predictedlabels)
f1=sklearn.metrics.f1_score(y_test, rectified_label,average='micro')
prec=sklearn.metrics.precision_score(y_test, rectified_label,average='weighted')
print(acc,nmi,f1,prec)
end=time.time()
print('time is',end-start)
%memit

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
0.9553166666666667 0.8899963652407529 0.9553166666666667 0.9554857101309593
time is 15.036395788192749
peak memory: 1301.29 MiB, increment: 0.58 MiB
